In [1]:
# Author Davide Aloi
# Scripts for the analyses of the EEG data (active task) collected at the Wellington Hospital - Aloi Davide PhD UoB

In [2]:
%matplotlib
import numpy as np
import mne
import os
import glob 
import cupy

mne.set_log_level("WARNING")
mne.cuda.init_cuda(verbose=True)
mne.utils.set_config('MNE_USE_CUDA', 'true')  

# Folders to the EEG active task data
output_folder = 'D:\\Raindrop_eeg_analysis\\p01\\'
sessions = ['D:\\Raindrop_data\\p01\\p01_w01\\eeg_baseline\\active_task\\', # baseline w 1
            'D:\\Raindrop_data\\p01\\p01_w02\\day04_eeg\\active_task\\', # post w 2
            'D:\\Raindrop_data\\p01\\p01_w03\\eeg_baseline\\active_task\\', # baseline w 3
            'D:\\Raindrop_data\\p01\\p01_w04\\day04_eeg\\active_task\\', # post w 4
            'D:\\Raindrop_data\\p01\\p01_w05\\eeg_baseline\\active_task\\', # baseline w5
            'D:\\Raindrop_data\\p01\\p01_w06\\day04_eeg\\active_task\\'] # post w6

# Paths to .mff files
sessions_raw = []
for n, session in enumerate(sessions):
    sessions_raw.append(glob.glob(session + '\*.mff')[0])

Using matplotlib backend: <object object at 0x000002BF20F21FC0>
Now using CUDA device 0
Enabling CUDA with 3.28 GB available memory


In [3]:
raw = mne.io.read_raw_egi(sessions_raw[0], preload=True)

In [4]:
montage = mne.channels.make_standard_montage('GSN-HydroCel-128')
raw_filtered = raw.copy().filter(1, 40)
picks_eeg = mne.pick_types(raw.info, meg=False, eeg=True, stim=False,
                       exclude='bads')
events = mne.find_events(raw, stim_channel='STI 014', verbose=True) # or STIM

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


72 events found
Event IDs: [1 2]


In [ ]:
event_id = dict(move=1,relax=2)
#event_id = dict(move=1)
# epoching ALL DATA without baseline correction just to use ransac (bad channels detection)
epoched = mne.Epochs(raw_filtered, events, event_id, 0, 2,
                baseline=(0, 0), reject=None,
                verbose=False, detrend=0, preload=True)


from autoreject import Ransac  # this uses ransac from PREP
ransac = Ransac(verbose=True, picks=picks_eeg, n_jobs=1)
epochs_clean = ransac.fit_transform(epoched)
bad_chs = ransac.bad_chs_ # list with bad channels according to RANSAC 


In [24]:
epoched

Number of events,5
Events,move: 3relax: 2
Time range,0.000 – 2.000 sec
Baseline,0.000 – 0.000 sec


In [ ]:

# Using autoreject to identify bad epochs
import autoreject
ar = autoreject.AutoReject(random_state=11,
                           n_jobs=1, verbose=True)
ar.fit(epoched[:12])  # fit on a few epochs to save time
epochs_ar, reject_log = ar.transform(epoched, return_log=True)

In [ ]:
ica = mne.preprocessing.ICA(random_state=99)
ica.fit(epoched[~reject_log.bad_epochs])

In [ ]:
#fig = mne.viz.plot_events(events, raw.info['sfreq'], event_id=event_id)
#fig.subplots_adjust(right=0.7)  # make room for legend
#raw_filtered.plot(events=events, start=5, duration=10, color='gray',
#         event_color={1: 'r', 2: 'g'})

In [ ]:
#raw.get_data(picks=picks_stim)